# Regression and Other Stories: National election study

*Note*: This code in this notebook follows the book order of presentation
Logistic regression, identifiability, and separation. See Chapters 13 and 14 in Regression and Other Stories.

In [1]:
import arviz as az
from bambi import Model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc3 as pm
from scipy import stats, special
import statsmodels.formula.api as smf

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
nes = pd.read_csv("https://raw.githubusercontent.com/avehtari/ROS-Examples/master/NES/data/nes.txt", sep=" ")
nes.head()

,year,resid,weight1,weight2,weight3,age,gender,race,educ1,urban,...,parent_party,white,year_new,income_new,age_new,vote.1,age_discrete,race_adj,dvote,rvote
536,1952,1,1.0,1.0,1.0,25,2,1,2,2.0,...,2.0,1,1,1,-2.052455,1.0,1,1.0,0.0,1.0
537,1952,2,1.0,1.0,1.0,33,2,1,1,2.0,...,0.0,1,1,1,-1.252455,1.0,2,1.0,1.0,0.0
538,1952,3,1.0,1.0,1.0,26,2,1,2,2.0,...,-2.0,1,1,0,-1.952455,1.0,1,1.0,0.0,1.0
539,1952,4,1.0,1.0,1.0,63,1,1,2,2.0,...,NaN,1,1,0,1.747545,1.0,3,1.0,0.0,1.0
540,1952,5,1.0,1.0,1.0,66,2,1,2,2.0,...,-2.0,1,1,-2,2.047545,1.0,4,1.0,0.0,1.0


In [3]:
ok = ((nes["year"]==1992) & nes["rvote"].notnull() & nes["dvote"].notnull()) & ((nes["rvote"]==1) | (nes["dvote"]==1))
nes92 = nes[ok]


# A single predictor logistic regression
### Logistic regression of vote preference on income

In [4]:
model = Model(nes92)
fit_1 = model.fit('rvote ~ income', family='bernoulli', link="logit", samples=1000, chains=4)

/opt/conda/lib/python3.8/site-packages/bambi/models.py:267: UserWarning: Modeling the probability that rvote=='1.0'
  warnings.warn(
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [income, Intercept]
INFO:pymc3:NUTS: [income, Intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.
INFO:pymc3:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.
The acceptance probability does not match the target. It is 0.88029259177819, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.
INFO:pymc3:The number of effective samples is smaller than 25% for some parameters.
/opt/conda/lib/python3.8/site-packages/arviz/data/io_pymc3.py:85: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


In [5]:
func_dict = {"Median": np.median,
            "MAD_SD":stats.median_abs_deviation,
             }
coefs = az.summary(fit_1, stat_funcs=func_dict, extend=False, round_to=2)
coefs

,Median,MAD_SD
Intercept[0],-1.40,0.13
income[0],0.32,0.04


### Predictions
Predict vote preference point estimate


In [18]:
new = 5

In [19]:
special.expit(coefs.loc["Intercept[0]","Median"] + coefs.loc["income[0]","Median"]*new)

0.549833997312478

Linear predictor with uncertainty

In [20]:
with model.backend.model:
    posterior_predictive = pm.sample_posterior_predictive(model.backend.trace, var_names=["Intercept", "income"])

In [21]:
linear_predictions = posterior_predictive["Intercept"] + posterior_predictive["income"]*new
linear_predictions.mean(), linear_predictions.std()

(0.22451521510392264, 0.12511689592812375)

Expected outcome with uncertainty

In [23]:
epred = special.expit(linear_predictions)
epred.mean(), epred.std()

(0.5556809949302626, 0.030780830593584378)

Predictive distribution for a new observation

TODO: This one is the posterior_predict method but not exactly sure what it is. Opened this issue to get some help https://github.com/avehtari/ROS-Examples/issues/15

### Prediction given a range of input values

In [ ]:
# TODO

# Fake data example